## 31. Event‑driven vs. request/response thinking

**Request/response**: caller waits for a direct reply (HTTP, RPC).  
**Event‑driven**: component emits an event; many listeners may react asynchronously.  

When to choose events:
• Loose coupling between producers and consumers.  
• Fan‑out (1 → N) or fan‑in (N → 1 aggregate) flows.

Watch‑outs:
• Harder to trace flow; need correlation IDs.  
• Delivery guarantees (at‑least‑once, exactly‑once) add complexity.

```text
Request/response:   UI → GET /balance → 200  $123.45
Event‑driven:       BankCore publishes 'BalanceChanged', accounting, email, mobile apps consume.
```

### Quick check

1. True / False Event producers know how many consumers will handle the event.

2. Which style is better for *real‑time UI updates* to many clients?
  a. request/response   b. event‑driven

<details><summary>Answer key</summary>

1. **False** — decoupled.
2. **b**.

</details>

## 32. Choosing sync vs. async API surfaces

**Sync** call blocks caller until result; simpler mental model.  
**Async** returns future/promise or uses callback/webhook.

Heuristics:
• If operation < 100 ms and caller can’t proceed without result → sync.  
• Long‑running, retry‑prone, or batch jobs → async with job ID + status poll/webhook.

Tip: design *both* – sync wrapper that internally polls async backend.

```python
# sync facade over async email send
def send_email_sync(client, msg):
    job = client.enqueue(msg)
    while not job.done():
        time.sleep(0.2)
    return job.result
```

### Quick check

1. Async APIs often return:
  a. raw value   b. job identifier

2. True / False Synchronous wrappers around async jobs violate REST.

<details><summary>Answer key</summary>

1. **b**.
2. **False** — pattern is common and acceptable.

</details>

## 33. Transaction boundaries & idempotency basics

A **transaction boundary** is the span where all changes commit atomically or roll back.  
Define them around *business invariants* (e.g., funds move from A to B).  

**Idempotent** operation: executing it once or many times has the same effect (PUT resource, deduplicated message IDs).  Crucial for retries in distributed systems.

```python
# idempotent debit using idempotency key
def debit(account, amount, key):
    if key in account.seen:
        return  # already applied
    with db.transaction():
        account.balance -= amount
        account.seen.add(key)
```

### Quick check

1. Executing an idempotent endpoint twice should:
  a. double effect   b. have single effect

2. True / False POST is idempotent by HTTP spec.

<details><summary>Answer key</summary>

1. **b**.
2. **False** — POST is not required to be idempotent.

</details>

## 34. Reading cyclomatic complexity & when to split code

**Cyclomatic complexity (CC)** counts independent paths through a function (#branches + 1).  
High CC (>10) → harder to test all paths, maintain.  Split large functions, extract policies into strategy objects.

Tools: `radon`, `ruff`, IDE plugins show CC in gutter.

```python
def rating(score, age, region):
    if region == 'EU':
        if age < 18:
            return 'deny'
        if score > 700:
            return 'gold'
    else:
        if score > 650 and age > 21:
            return 'gold'
    return 'standard'  # CC = 5 paths
```

### Quick check

1. High CC means:
  a. few test cases needed  b. many test cases needed

2. True / False Early returns can *reduce* CC.

<details><summary>Answer key</summary>

1. **b**.
2. **True** — simplify branching.

</details>

## 35. Code smells checklist

Common **smells** indicating future pain:
• **Long function / class** – does too much.  
• **Primitive obsession** – raw strings/ints instead of rich types.  
• **Shotgun surgery** – single change requires edits across many files.  
• **Feature envy** – method uses data from another object more than its own.  
• **Duplicated code** – violates DRY.  

Smells aren’t bugs; they’re *warning signals* prompting refactor.

```text
Smell: Primitive obsession
    price = 1999  # cents? dollars? unclear

Refactor:
    class Money(NamedTuple):
        amount: int  # cents
    price = Money(1999)
```

### Quick check

1. Shotgun surgery indicates:
  a. low coupling   b. high coupling

2. True / False Duplicated code is harmless if tests pass.

<details><summary>Answer key</summary>

1. **b**.
2. **False** – duplication increases maintenance cost.

</details>